### 3108. Minimum Cost Walk in Weighted Graph
There is an undirected weighted graph with n vertices labeled from 0 to n - 1.

You are given the integer n and an array edges, where edges[i] = [ui, vi, wi] indicates that there is an edge between vertices ui and vi with a weight of wi.

A walk on a graph is a sequence of vertices and edges. The walk starts and ends with a vertex, and each edge connects the vertex that comes before it and the vertex that comes after it. It's important to note that a walk may visit the same edge or vertex more than once.

The cost of a walk starting at node u and ending at node v is defined as the bitwise AND of the weights of the edges traversed during the walk. In other words, if the sequence of edge weights encountered during the walk is w0, w1, w2, ..., wk, then the cost is calculated as w0 & w1 & w2 & ... & wk, where & denotes the bitwise AND operator.

You are also given a 2D array query, where query[i] = [si, ti]. For each query, you need to find the minimum cost of the walk starting at vertex si and ending at vertex ti. If there exists no such walk, the answer is -1.

Return the array answer, where answer[i] denotes the minimum cost of a walk for query i.

In [9]:
# Key points to note:
# 1. A walk is different from a path, a walk means we can backtrack 
# 2. The prompt indicates multiple edges between a vertex.
# 3. Bitwise AND operation would not result in a number that is larger than either of the two they are performing the operation on.
#
# Intuition:
# Use union find to find all the connected components in the graph.
# Use the key point #3 & 1 - the minimum cost of a walk between two connected vertex is therefore the cost of the component they belong to. 
# ie; if we can get from v1 to v2, we'd just make them walk the entire graph in the worst case to minimize the cost by ANDing as many edges as possible.

from typing import List

class UnionFind:
    def __init__(self, n:int) -> None:
        self.parent = list(range(n))
        self.size = [1]*n #How many nodes in this tree, uf optimization
    def find(self, x:int) -> int:
        if x != self.parent[x]: #If x is not it's own parent, we travel up the tree to find the element that is its own parent
            self.parent[x] = self.find(self.parent[x]) #updating to point every node to the tree's root
        return self.parent[x]
    def union(self, x:int, y:int) ->int:
        rootx = self.find(x)
        rooty = self.find(y)
        if rootx != rooty:
            if self.size[rootx] > self.size[rooty]: #we put the smaller tree under the bigger tree. ie: make the smaller tree's root the bigger tree's root
                self.parent[rooty] = rootx
                self.size[rootx] += self.size[rooty]
            else:
                self.parent[rootx] = rooty
                self.size[rooty] += self.size[rootx]

def minimumCost(n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:
    #1. Build graph
    uf = UnionFind(n)
    for v1, v2, w in edges:
        uf.union(v1,v2)
        
    #2. Build hashmap to track component - walk cost
    comp_cost = {}
    for v1, v2, w in edges:
        root = uf.find(v1)
        if root not in comp_cost:
            comp_cost[root] = w
        else:
            comp_cost[root] &= w
    
    #3. Answer the queries - if v1 and v2 are in the same component, the minimum walk cost is the component walk cost
    res = []
    for s, e in query:
        roots = uf.find(s)
        roote = uf.find(e)
        if roots != roote:
            res.append(-1)
        else:
            res.append(comp_cost[roots])
    return res
            
        

In [11]:
n = 5
edges = [[0,1,7],[1,3,7],[1,2,1]]
query = [[0,3],[3,4]]

assert minimumCost(n, edges, query) == [1, -1], "Wrong Answer"